In [2]:
import cv2
import os
from tkinter import Tk, Button, Label, OptionMenu, StringVar, Entry
from PIL import Image, ImageTk, ImageEnhance, ImageFilter
import hashlib
from random import randint

if not os.path.exists('dataset'):
    os.makedirs('dataset')
    print("Created directory: dataset")

cap = None
camera_index = 0  
image_id = 1
save_directory = 'dataset'  

def get_camera_indexes():
    indexes = []
    index = 0
    while True:
        cap = cv2.VideoCapture(index)
        if not cap.read()[0]:
            break
        indexes.append(index)
        cap.release()
        index += 1
    return indexes

def apply_augmentations(frame):
    augmented_images = []
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    augmented_images.append(img)
    for angle in [90, 180, 270]:
        rotated = img.rotate(angle)
        augmented_images.append(rotated)
    zoomed_in = img.resize((int(img.width * 1.5), int(img.height * 1.5)))
    augmented_images.append(zoomed_in)
    zoomed_out = img.resize((int(img.width * 0.5), int(img.height * 0.5)))
    augmented_images.append(zoomed_out)
    blurred = img.filter(ImageFilter.GaussianBlur(radius=5))
    augmented_images.append(blurred)
    left = randint(0, 20)
    upper = randint(0, 20)
    right = img.width - randint(0, 20)
    lower = img.height - randint(0, 20)
    cropped = img.crop((left, upper, right, lower))
    augmented_images.append(cropped)
    flipped_horizontal = img.transpose(Image.FLIP_LEFT_RIGHT)
    augmented_images.append(flipped_horizontal)
    
    flipped_vertical = img.transpose(Image.FLIP_TOP_BOTTOM)
    augmented_images.append(flipped_vertical)
    for factor in [0.5, 1.5]:  
        brightness_enhancer = ImageEnhance.Brightness(img)
        enhanced = brightness_enhancer.enhance(factor)
        augmented_images.append(enhanced)

    return augmented_images

def capture_image(event=None):
    global image_id
    print("Capture image function called.")

    ret, frame = cap.read()

    if ret:
        hash_object = hashlib.md5(frame.tobytes())
        hash_str = hash_object.hexdigest()
        
        augmented_images = apply_augmentations(frame)

        for i, img in enumerate(augmented_images):
            image_name = f"IMG_{image_id:04d}_JPG.rf.{hash_str}_{i}.jpg"
            image_path = os.path.join(save_directory, image_name)

            img.save(image_path)
            print(f'Saved image to {image_path}')

        print(f'Image set for ID {image_id} captured and saved.')

        image_id += 1
    else:
        print("Failed to capture frame from camera.")

def select_camera():
    global cap, camera_index
    camera_index = int(camera_var.get())
    
    if cap is not None:
        cap.release()  

    update_camera_label()

def update_camera_label():
    camera_name = f'Camera ID: {camera_index}'
    camera_label.config(text=camera_name)

def start_camera():
    global cap
    cap = cv2.VideoCapture(camera_index)  

    if not cap.isOpened():
        print(f"Error: Unable to access camera {camera_index}.")
        return

    def update_frame():
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            camera_display.imgtk = imgtk  
            camera_display.config(image=imgtk)
        else:
            print("Error: Unable to read frame from camera.")

        camera_display.after(10, update_frame)  

    update_frame()  

def set_directory():
    global save_directory
    save_directory = dir_entry.get()
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
        print(f"Created directory: {save_directory}")

root = Tk()
root.title("Camera Capture Tool")

camera_indexes = get_camera_indexes()

if camera_indexes:
    camera_var = StringVar(value=camera_indexes[0])
    camera_dropdown = OptionMenu(root, camera_var, *camera_indexes, command=lambda _: select_camera())
    camera_dropdown.pack()
    camera_label = Label(root, text=f'Camera ID: {camera_indexes[0]}')
    camera_label.pack()
    start_camera_button = Button(root, text="Start Camera", command=start_camera)
    start_camera_button.pack()
    camera_display = Label(root)
    camera_display.pack()
    capture_button = Button(root, text="Capture Image", command=capture_image)
    capture_button.pack()
    
    dir_entry_label = Label(root, text="Save Directory:")
    dir_entry_label.pack()
    dir_entry = Entry(root)
    dir_entry.pack()
    set_dir_button = Button(root, text="Set Directory", command=set_directory)
    set_dir_button.pack()
    
    root.bind('<F2>', capture_image)
    
else:
    Label(root, text="No cameras found.").pack()

root.mainloop()


Created directory: 1
Capture image function called.
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_0.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_1.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_2.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_3.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_4.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_5.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_6.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_7.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_8.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_9.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_10.jpg
Saved image to 1\IMG_0001_JPG.rf.c42c4d058c4204b3307259873468fbd5_11.jpg
Image set for ID 1 captured and saved.
Capture image function called.
Saved image 